In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-634d696d-15fe-4e5f-926d-b3e246fa0af4;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 618ms :: artifacts dl 56ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [2]:
df = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://marise-3cco/vendas_anuais.csv')

22/05/25 21:48:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
df.show()

+----------+------------+--------------+------+
|id_cliente|total_vendas|numero_pedidos|perfil|
+----------+------------+--------------+------+
|    100001|       80064|             3|     F|
|    100002|       21753|             3|     F|
|    100003|        7458|             2|     M|
|    100004|        4986|             3|     M|
|    100005|       72311|             4|     F|
|    100006|        6943|             2|     F|
|    100007|        4015|             2|     M|
|    100008|        5861|             2|     M|
|    100009|       36463|             2|     F|
|    100010|        4431|             2|     M|
|    100011|       21641|             1|     F|
|    100012|       15792|             2|     F|
|    100013|       28958|             1|     M|
|    100014|       10444|             7|     M|
|    100015|         823|             3|     M|
|    100016|        9021|             2|     M|
|    100017|        5909|             2|     M|
|    100018|       36138|             1|

In [12]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

_df = df.select(
    col('id_cliente').cast(IntegerType()).alias('ID_CLIENTE'),
    col('total_vendas').cast(IntegerType()).alias('TOTAL_VENDAS'),
    col('numero_pedidos').cast(IntegerType()).alias('NUMERO_PEDIDOS'),
    col('perfil').alias('PERFIL')
)

_df.write \
   .mode('overwrite') \
   .csv('s3a://marise-3cco/vendas_anuais')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://bd-dw-01.database.windows.net;databaseName=BD-DW;') \
 .option('dbtable', 'VENDAS_ANUAIS') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()

22/05/23 01:28:49 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/23 01:28:50 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
